In [1]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 117.9 MB/s eta 0:00:00


In [2]:
!pip install transformers accelerate sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 81.1 MB/s eta 0:00:00


In [3]:
pdf_path = "/content/Neural_Additative_Models.pdf"

In [4]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 10.0 MB/s eta 0:00:00


In [5]:

import fitz
from tqdm.auto import tqdm

def text_formatter(text: str) -> str:
    """Performs minor formatting on text."""
    cleaned_text = text.replace("\n", " ").strip()
    return cleaned_text

# Open PDF and get lines/pages
# Note: this only focuses on text, rather than images/figures etc
def open_and_read_pdf(pdf_path: str) -> list[dict]:
    """
    Opens a PDF file, reads its text content page by page, and collects statistics.

    Parameters:
        pdf_path (str): The file path to the PDF document to be opened and read.

    Returns:
        list[dict]: A list of dictionaries, each containing the page number
        (adjusted), character count, word count, sentence count, token count, and the extracted text
        for each page.
    """
    doc = fitz.open(pdf_path)  # open a document
    pages_and_texts = []
    for page_number, page in tqdm(enumerate(doc)):  # iterate the document pages
        text = page.get_text()  # get plain text encoded as UTF-8
        text = text_formatter(text)
        pages_and_texts.append({"page_number": page_number - 41,  # adjust page numbers since our PDF starts on page 42
                                "page_char_count": len(text),
                                "page_word_count": len(text.split(" ")),
                                "page_sentence_count_raw": len(text.split(". ")),
                                "page_token_count": len(text) / 4,
                                "text": text})
    return pages_and_texts

pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:2]

0it [00:00, ?it/s]

[{'page_number': -41,
  'page_char_count': 3031,
  'page_word_count': 435,
  'page_sentence_count_raw': 20,
  'page_token_count': 757.75,
  'text': 'Neural Additive Models: Interpretable Machine Learning with Neural Nets Rishabh Agarwal ∗ Google Research, Brain Team Levi Melnick Microsoft Research Nicholas Frosst Cohere Xuezhou Zhang University of Wisconsin-Madison Ben Lengerich MIT Rich Caruana Microsoft Research Geoffrey E. Hinton Google Research, Brain Team Abstract Deep neural networks (DNNs) are powerful black-box predictors that have achieved impressive performance on a wide variety of tasks. However, their accuracy comes at the cost of intelligibility: it is usually unclear how they make their decisions. This hinders their applicability to high stakes decision-making domains such as healthcare. We propose Neural Additive Models (NAMs) which combine some of the expressivity of DNNs with the inherent intelligibility of generalized additive models. NAMs learn a linear combination o

In [6]:
import random

random.sample(pages_and_texts, k=3)

[{'page_number': -31,
  'page_char_count': 3448,
  'page_word_count': 464,
  'page_sentence_count_raw': 70,
  'page_token_count': 862.0,
  'text': 'References [1] Julia Angwin, Jeff Larson, Lauren Kirchner, and Surya Mattu. Machine Bias: There’s software used across the country to predict future criminals. And it’s biased against blacks, 2016. URL https://www.propublica.org/article/ machine-bias-risk-assessments-in-criminal-sentencing. [Accessed Febru- ary 1, 2020]. [2] Devansh Arpit, Stanislaw Jastrzkebski, Nicolas Ballas, David Krueger, Emmanuel Bengio, Maxinder S Kanwal, Tegan Maharaj, Asja Fischer, Aaron Courville, Yoshua Bengio, et al. A closer look at memorization in deep networks. ICML, 2017. [3] Richard Berk, Hoda Heidari, Shahin Jabbari, Michael Kearns, and Aaron Roth. Fairness in criminal justice risk assessments: The state of the art. Sociological Methods & Research, 2018. [4] Rich Caruana. Multitask learning. Machine Learning, 1997. [5] Rich Caruana, Yin Lou, Johannes Gehrk

In [7]:
import pandas as pd

df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,-41,3031,435,20,757.75,Neural Additive Models: Interpretable Machine ...
1,-40,3779,619,40,944.75,"In this paper, we make restrictions on the str..."
2,-39,3936,650,25,984.00,−1.00 −0.75 −0.50 −0.25 0.00 0.25 0.50 0.75 1....
3,-38,4593,791,36,1148.25,20 40 60 80 100 Input (xi) −1.0 −0.5 0.0 0.5 1...
4,-37,4138,685,24,1034.50,Table 1: Single-task learning NAM results. Mea...


In [8]:
# Get stats
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count
count,23.00,23.00,23.00,23.00,23.00
mean,-30.00,3207.39,530.74,25.09,801.85
std,6.78,1270.39,202.98,17.25,317.60
min,-41.00,286.00,45.00,3.00,71.50
25%,-35.50,2589.00,441.00,14.00,647.25
50%,-30.00,3448.00,558.00,24.00,862.00
75%,-24.50,4037.00,667.50,29.50,1009.25
max,-19.00,4962.00,844.00,70.00,1240.50


In [9]:
import fitz  # PyMuPDF
import re

def extract_citations(pdf_path):
    """
    Extracts the raw reference entries from a PDF research paper.
    Returns a list of citation strings.
    """
    # Read PDF text
    doc = fitz.open(pdf_path)
    full_text = ""
    for page in doc:
        full_text += page.get_text("text")

    # Normalize whitespace
    text = re.sub(r'\s+', ' ', full_text)

    # Find the References section
    ref_start = re.search(r'\b(references|bibliography)\b', text, re.IGNORECASE)
    if not ref_start:
        print("No References section found")
        return []

    refs_text = text[ref_start.end():]

    # Stop if Appendix, Acknowledgements, etc. appear
    stop_match = re.search(r'\b(appendix|acknowledg(e)?ments|supplementary|figures?|tables?)\b', refs_text, re.IGNORECASE)
    if stop_match:
        refs_text = refs_text[:stop_match.start()]

    # Split into individual references
    # Works with numbered and unnumbered formats
    citations = re.split(r'(?=\[\d+\]|^\d+\.\s+|(?<=\.)\s*[A-Z][^,]+\(\d{4}\))', refs_text)
    citations = [c.strip() for c in citations if len(c.strip()) > 40]

    return citations

citations = extract_citations(pdf_path)

print(f"\nExtracted {len(citations)} citations:\n")
for i, c in enumerate(citations, 1):
    print(f"{i}. {c}\n")



Extracted 46 citations:

1. [1] Julia Angwin, Jeff Larson, Lauren Kirchner, and Surya Mattu. Machine Bias: There’s software used across the country to predict future criminals. And it’s biased against blacks, 2016. URL https://www.propublica.org/article/ machine-bias-risk-assessments-in-criminal-sentencing. [Accessed Febru- ary 1, 2020].

2. [2] Devansh Arpit, Stanislaw Jastrzkebski, Nicolas Ballas, David Krueger, Emmanuel Bengio, Maxinder S Kanwal, Tegan Maharaj, Asja Fischer, Aaron Courville, Yoshua Bengio, et al. A closer look at memorization in deep networks. ICML, 2017.

3. [3] Richard Berk, Hoda Heidari, Shahin Jabbari, Michael Kearns, and Aaron Roth. Fairness in criminal justice risk assessments: The state of the art. Sociological Methods & Research, 2018.

4. [4] Rich Caruana. Multitask learning. Machine Learning, 1997.

5. [5] Rich Caruana, Yin Lou, Johannes Gehrke, Paul Koch, Marc Sturm, and Noemie Elhadad. Intelligible models for healthcare: Predicting pneumonia risk and ho

In [10]:
!pip install mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.8/442.8 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 kB 19.7 MB/s eta 0:00:00


In [11]:
import re
from mistralai import Mistral

# Initialize Mistral client
api_key = "API Key"  # Replace with your key
model = "mistral-small-latest"    # You can use mistral-medium or mistral-large for better results
client = Mistral(api_key=api_key)

def extract_title_from_citation(citation: str) -> str:
    """
    Extracts the title from a citation string using regex first,
    and falls back to a Mistral model if regex fails.
    """
    # Simple regex patterns for common formats (APA/MLA/IEEE)
    patterns = [
        r'“([^”]+)”',             # Titles in quotation marks
        r'\"([^\"]+)\"',          # Titles in regular double quotes
        r'‘([^’]+)’',             # Titles in single quotes
        r'\'([^\']+)\'',          # Titles in regular single quotes
        r'\.\s*([A-Z][^.]+)\.\s', # APA-like (Author. Title. Journal...)
    ]

    for pattern in patterns:
        match = re.search(pattern, citation)
        if match:
            return match.group(1).strip()

    # Fall back to Mistral if regex fails
    prompt = f"""
    Extract only the TITLE of the research paper from this citation:
    Citation: {citation}
    Return only the title.
    """
    response = client.chat.complete(
        model=model,
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )
    # Access content attribute directly
    return response.choices[0].message.content.strip()


titles = [extract_title_from_citation(c) for c in citations]

for i, t in enumerate(titles):
    print(f"{i+1}. {t}")

1. Machine Bias: There’s software used across the country to predict future criminals
2. A closer look at memorization in deep networks
3. Fairness in criminal justice risk assessments: The state of the art
4. Multitask learning
5. Intelligible models for healthcare: Predicting pneumonia risk and hospital 30-day readmission
6. XGBoost: A scalable tree boosting system
7. Adaptive machine learning for credit card fraud detection
8. The accuracy, fairness, and limits of predicting recidivism
9. FICO Explainable Machine Learning Challenge
10. Understanding the difﬁculty of training deep feedforward neural networks
11. Google vizier: A service for black-box optimization
12. Generalized linear and generalized additive models in studies of species distributions: setting the scene
13. Generalized additive models for medical research
14. Generalized Additive Models
15. A case study of hurdle and generalized additive models in astronomy: the escape of ionizing radiation
16. Delving deep into rec

In [12]:
!pip install requests tqdm

In [13]:
!pip install feedparser requests tqdm

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 4.9 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=4f921c6b0a03b9ebe2ad3b640946ff218f7280b80c962a98f4382bd9b40e8814
  Stored in directory: /root/.cache/pip/wheels/03/f5/1a/23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k


In [14]:
import urllib.parse
import feedparser
from tqdm import tqdm

ARXIV_API_URL = "http://export.arxiv.org/api/query"

def get_arxiv_pdf_link(title):
    """
    Search arXiv for a paper by title and return the PDF link if found.
    """
    # Build query
    query = f"ti:\"{title}\""
    params = {
        "search_query": query,
        "start": 0,
        "max_results": 1
    }
    query_str = urllib.parse.urlencode(params)
    url = f"{ARXIV_API_URL}?{query_str}"

    # Parse feed
    feed = feedparser.parse(url)
    if not feed.entries:
        return None

    # Get the first entry and its PDF link
    entry = feed.entries[0]
    for link in entry.links:
        # Check for PDF links by type or relation
        if link.type == "application/pdf" or link.rel == "alternate":
            return link.href
    return None

def citations_to_arxiv_links(citations):
    """
    Convert a list of citations (titles) to arXiv PDF URLs if available.
    """
    results = {}
    for c in tqdm(citations, desc="Searching arXiv"):
        results[c] = get_arxiv_pdf_link(c)
    return results

if __name__ == "__main__":
    citations = [
        "Attention Is All You Need",
        "Deep Residual Learning for Image Recognition",
        "A Survey on Graph Neural Networks"
    ]

    links = citations_to_arxiv_links(titles)

    print("\n Results:")
    for cite, link in links.items():
        if link:
            print(f"\033[92m{cite} → {link}\033[0m")
        else:
            print(f"\033[31m{cite} → No arXiv PDF found\033[0m")

Searching arXiv: 100%|██████████| 46/46 [00:12<00:00,  3.80it/s]


 Results:
Machine Bias: There’s software used across the country to predict future criminals → No arXiv PDF found
A closer look at memorization in deep networks → http://arxiv.org/abs/1706.05394v2
Fairness in criminal justice risk assessments: The state of the art → http://arxiv.org/abs/1703.09207v2
Multitask learning → http://arxiv.org/abs/1510.00633v1
Intelligible models for healthcare: Predicting pneumonia risk and hospital 30-day readmission → No arXiv PDF found
XGBoost: A scalable tree boosting system → http://arxiv.org/abs/1603.02754v3
Adaptive machine learning for credit card fraud detection → No arXiv PDF found
The accuracy, fairness, and limits of predicting recidivism → No arXiv PDF found
FICO Explainable Machine Learning Challenge → No arXiv PDF found
Understanding the difﬁculty of training deep feedforward neural networks → No arXiv PDF found
Google vizier: A service for black-box optimization → No arXiv PDF found
Generalized linear and generalized additive models in studi

In [15]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [16]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 123.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 112.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 134.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 14.2 MB/

In [19]:
import fitz
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
import requests
from google.colab import userdata # Import userdata

# Splitter and embeddings
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

embedding_model_name = "all-MiniLM-L6-v2"
embedder = HuggingFaceEmbeddings(model_name=embedding_model_name)


all_chunks = []

# Ensure pdf_links is defined, even if empty
if 'links' in globals() and links:
    # Filter out None values and get the valid arXiv links
    pdf_links = [link for link in links.values() if link]
else:
    pdf_links = [] # Initialize as empty list if no links found or 'links' is not defined

# Process remote PDFs (Arxiv)
for link in pdf_links:
    try:
        response = requests.get(link)
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
        with fitz.open(stream=response.content, filetype="pdf") as doc:
            text = "".join(page.get_text() for page in doc)
        chunks = text_splitter.split_text(text)
        all_chunks.extend(chunks)
        print(f"\033[92mProcessed remote PDF: {link}\033[0m")
    except requests.exceptions.RequestException as e:
        print(f"\033[31mFailed to download or process remote PDF {link}: {e}\033[0m")
    except Exception as e:
        print(f"\033[31An error occurred while processing remote PDF {link}: {e}\033[0m")


# Process local PDF stored in Colab
local_path = pdf_path  # 👈 path to your uploaded file
try:
    with fitz.open(local_path) as doc:
        local_text = "".join(page.get_text() for page in doc)
    local_chunks = text_splitter.split_text(local_text)
    all_chunks.extend(local_chunks)
    print(f"\033[92mProcessed local PDF: {local_path}\033[0m")
except FileNotFoundError:
    print(f"\033[31mLocal PDF not found at {local_path}. Please upload the file.\033[0m")
except Exception as e:
    print(f"\033[31mAn error occurred while processing local PDF {local_path}: {e}\033[0m")


# Create or update vector DB
if all_chunks:
    db = Chroma.from_texts(all_chunks, embedder)
    print("\033[92mIndexed {len(all_chunks)} chunks (including original paper).\033[0m")
else:
    print("Skipping vector DB creation as no chunks were processed.")

/tmp/ipython-input-2368158485.py:13: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = HuggingFaceEmbeddings(model_name=embedding_model_name)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Processed remote PDF: http://arxiv.org/abs/1706.05394v2
Processed remote PDF: http://arxiv.org/abs/1703.09207v2
Processed remote PDF: http://arxiv.org/abs/1510.00633v1
Processed remote PDF: http://arxiv.org/abs/1603.02754v3
Processed remote PDF: http://arxiv.org/abs/1506.03850v2
Processed remote PDF: http://arxiv.org/abs/1805.07435v2
Processed remote PDF: http://arxiv.org/abs/1512.03385v1
Processed remote PDF: http://arxiv.org/abs/1412.6980v9
Processed remote PDF: http://arxiv.org/abs/2505.08610v1
Processed remote PDF: http://arxiv.org/abs/2201.03092v1
Processed remote PDF: http://arxiv.org/abs/1806.08734v3
Processed remote PDF: http://arxiv.org/abs/1602.04938v3
Processed remote PDF: http://arxiv.org/abs/1811.10154v3
Processed remote PDF: http://arxiv.org/abs/2103.11251v2
Processed remote PDF: http://arxiv.org/abs/1206.2944v2
Processed remote PDF: http://arxiv.org/abs/2006.10739v1
Processed remote PDF: http://arxiv.org/abs/2104.08903v1
Processed local PDF: /content/Neural_Additative_Mo

In [22]:
from transformers import pipeline

# Mistral AI model
generator = pipeline(
    "text-generation",
    model="mistralai/Mistral-7B-v0.1",
    max_new_tokens=200
)


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Device set to use cuda:0


In [26]:
from langchain import hub
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss

query = "What is a Neural Additative Model?"

# Re-initialize the SentenceTransformer embedder
embedder_st = SentenceTransformer("all-MiniLM-L6-v2")


# Retrieve top 2 docs
retrieved_docs_objects = db.similarity_search_with_score(query, k=2) # Use similarity_search_with_score
retrieved_docs = [doc.page_content for doc, score in retrieved_docs_objects] # Extract text content and ignore score

# Build prompt
context = "\n".join(retrieved_docs)
prompt = f"""
You are an expert AI research assistant.
Your task is to answer the user's question using the provided research excerpts.
Base your reasoning ONLY on the context.
When relevant, highlight key findings or methods from the text.

Context:
{context}

QUESTION:
{query}

---
ANSWER:


"""

# Generate with pipeline (no API call)
response = generator(prompt)[0]["generated_text"]
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



You are an expert AI research assistant.
Your task is to answer the user's question using the provided research excerpts.
Base your reasoning ONLY on the context.
When relevant, highlight key findings or methods from the text.

Context:
∗Correspondence
to:
Rishabh
Agarwal
<rishabhagarwal@google.com>,
Levi
Melnick
<lemeln@microsoft.com>, and Rich Caruana <rcaruana@microsoft.com>.
2Linear models, shallow decision trees and GAMs are interpretable only if the features they are trained on
are interpretable.
35th Conference on Neural Information Processing Systems (NeurIPS 2021).
arXiv:2004.13912v2  [cs.LG]  24 Oct 2021
In this paper, we make restrictions on the structure of neural networks, which yields a family of
glass-box models called Neural Additive Models (NAMs), that are inherently interpretable while
suffering little loss in prediction accuracy when applied to tabular data. Methodologically, NAMs
belong to a model family called Generalized Additive Models (GAMs) [14]. GAMs have the